__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/wiktionary_wortschatz/blob/master/1_wiktionary_parsen/x10_d00_wiktionary_de_parser.ipynb)__

# wiktionary_de_parser
* erstellt die Datei wiktionary_vvv.pickle in data_00
* wendet wiktionary_de_parser an
* braucht ca. 15 Minuten

### Doku
* page_id <=> title eineindeutig
* page_part ist die Abschnittsnummer der Seite
* section_id2 identifiziert page_id-page_part-Kombination (heißt später lemma_id)

## Import

In [50]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['in_colab']     = True
Start Time: 16:00:27


time: 47.6 ms (started: 2023-02-15 16:00:27 +00:00)


In [51]:
# alle anderen
try:
    import pandasklar as pak, bj_nlp, wiktionary_de_parser, wget, bz2file

except ImportError as e:
    !pip install pandasklar bj_nlp wiktionary_de_parser wget bz2file
    import pandasklar as pak, bj_nlp, wiktionary_de_parser, wget, bz2file  
    
grid       = pak.grid
check_mask = pak.check_mask        


time: 80.2 ms (started: 2023-02-15 16:00:28 +00:00)


## Einstellungen

In [52]:
# Was debuggen?
suche_debug = ['entlassen']

# Load
verzeichnis_load          = 'data_00'
bzfile_url                = 'https://dumps.wikimedia.org/dewiktionary/latest/'
bzfile_filenam            = 'dewiktionary-latest-pages-articles-multistream.xml.bz2'
bzfile_filename           = os.path.join( verzeichnis_load, bzfile_filenam ) 

# Was parsen?      
min_id                    = 0         # 81: Mensch
max_id                    = 999999999 #22600 
version                   = '_all'
#version                  = '_short'


# Save
verzeichnis_save          = 'data_00'
wiktionary_filename       = os.path.join( verzeichnis_save, 'wiktionary'       +version+'.pickle' )       
wiktionary_trash_filename = os.path.join( verzeichnis_save, 'wiktionary_trash' +version+'.pickle' )  

time: 47.9 ms (started: 2023-02-15 16:00:28 +00:00)


In [53]:
# verbose
pak.Config.set('VERBOSE', True)

VERBOSE = True
--> setting verbose=True as default for all pandasklar functions

time: 47.6 ms (started: 2023-02-15 16:00:28 +00:00)


## Vorbereiten

In [54]:
# data_00 erzeugen
os.makedirs(verzeichnis_load, exist_ok=True) 

time: 48.7 ms (started: 2023-02-15 16:00:28 +00:00)


In [55]:
# download bzfile if necessary
# 1 min

if not os.path.isfile(bzfile_filename):
    filename = wget.download(bzfile_url + bzfile_filenam, out=verzeichnis_load)
else:
    print(bzfile_filenam, 'existiert bereits')
    
assert os.path.isfile(bzfile_filename)

dewiktionary-latest-pages-articles-multistream.xml.bz2 existiert bereits
time: 45 ms (started: 2023-02-15 16:00:28 +00:00)


In [56]:
# Vorbereiten
bz          = bz2file.BZ2File(bzfile_filename)  # XML-Quelldatei
result_list = []                                # nimmt die geparsten Einträge auf
config      = {"include_wikitext": True}

time: 51.3 ms (started: 2023-02-15 16:00:28 +00:00)


## Parsen

In [57]:
# Parsen

title_last     = ''
page_part_last = 0
page_base      = 0

for num, record in enumerate(  wiktionary_de_parser.Parser(bz, config=config)  ):
    
    # num:    ID des Eintrags
    # record: Der Eintrag selbst
    
    # title
    if 'title' in record:
        title = record['title']
    else:
        title = ''
        print('no title')
        
    # page_id und page_part    
    if title == title_last:
        page_id   = page_base
        page_part = page_part_last + 1
    else:
        page_id   = num   # fortlaufende Nummer
        page_base = num
        page_part = 0
        
    
    # Am Anfang was auslassen?
    if page_id < min_id:
        continue        
       
    # Vorzeitig abbrechen?    
    if page_id > max_id:
        break
    
    # Fremdsprachiges überspringen
    if 'lang_code' in record   and   record['lang_code'] != 'de': 
        continue
    if 'lang' in record   and   record['lang'] != 'Deutsch': 
        continue        
    
    # record ergänzen
    record['page_id']     = page_id
    record['page_part']   = page_part    
    
    # last
    title_last     = title
    page_part_last = page_part    
    
    # Statusanzeige
    if page_id % 1000 == 0:    
        clear_output()
        print( page_id, title )
    
    # fürs Debugging aufbewahren
    #if num in catch_debug:
    #    debug_list.append(record)
    
    # Ausgabe
    result_list.append(record)
   
    

1118000 Sebaldinen
time: 16min 45s (started: 2023-02-15 16:00:28 +00:00)


## In Dataframe wandeln

In [58]:
# In DataFrame wandeln
wiktionary = pak.dataframe(result_list,verbose=False)

time: 3.1 s (started: 2023-02-15 16:17:14 +00:00)


In [59]:
# Daten ergänzen
wiktionary['section_len'] = wiktionary.wikitext.str.len() # Volumen der Seite
wiktionary['error'] = ''

time: 1.21 s (started: 2023-02-15 16:17:17 +00:00)


In [60]:
# Spalten sortieren
wiktionary = pak.move_cols( wiktionary, ['page_id','page_part','title','lemma'])
wiktionary = pak.move_cols( wiktionary, ['section_len','flexion','ipa','syllables','lang','lang_code','wikitext','error'], to=-1)

time: 1.5 s (started: 2023-02-15 16:17:18 +00:00)


In [58]:
grid(wiktionary,pak.sample)

12 rows out of 699


time: 357 ms


In [62]:
# suche_debug 
mask = wiktionary.lemma.isin(suche_debug)
grid(wiktionary[mask].sort_values(['page_id',]))

19 rows


,page_id,page_part,title,lemma,inflected,rhymes,pos,section_len,flexion,ipa,syllables,lang,lang_code,wikitext,error
14147,22703,0,entlassen,entlassen,False,[asn̩],{'Verb': []},4961,"{'Präsens_ich': 'entlasse', 'Präsens_du': 'ent...",[ɛntˈlasn̩],"[ent, las, sen]",Deutsch,de,=== {{Wortart|Verb|Deutsch}} ===\n\n{{Deutsch ...,
41011,108337,0,entließ,entlassen,True,[iːs],{'Verb': ['Konjugierte Form']},470,NaN,[ɛntˈliːs],"[ent, ließ]",Deutsch,de,=== {{Wortart|Konjugierte Form|Deutsch}} ===\n...,
62481,139661,0,entlasse,entlassen,True,[asə],{'Verb': ['Konjugierte Form']},614,NaN,[ɛntˈlasə],"[ent, las, se]",Deutsch,de,=== {{Wortart|Konjugierte Form|Deutsch}} ===\n...,
75724,162009,0,entlässt,entlassen,True,[ɛst],{'Verb': ['Konjugierte Form']},482,NaN,[ˌɛntˈlɛst],"[ent, lässt]",Deutsch,de,=== {{Wortart|Konjugierte Form|Deutsch}} ===\n...,
224960,394049,0,entließe,entlassen,True,[iːsə],{'Verb': ['Konjugierte Form']},478,NaN,[ɛntˈliːsə],"[ent, lie, ße]",Deutsch,de,=== {{Wortart|Konjugierte Form|Deutsch}} ===\n...,
275572,454733,0,entlasst,entlassen,True,[ast],{'Verb': ['Konjugierte Form']},475,NaN,[ɛntˈlast],"[ent, lasst]",Deutsch,de,=== {{Wortart|Konjugierte Form|Deutsch}} ===\n...,
275576,454737,0,entlassend,entlassen,True,[asn̩t],{'Adjektiv': ['Partizip I']},314,NaN,[ɛntˈlasn̩t],"[ent, las, send]",Deutsch,de,=== {{Wortart|Partizip I|Deutsch}} ===\n\n{{Wo...,
275579,454740,0,entließet,entlassen,True,[iːsət],{'Verb': ['Konjugierte Form']},407,NaN,[ˌɛntˈliːsət],"[ent, lie, ßet]",Deutsch,de,=== {{Wortart|Konjugierte Form|Deutsch}} ===\n...,
275580,454741,0,entließen,entlassen,True,[iːsn̩],{'Verb': ['Konjugierte Form']},636,NaN,[ˌɛntˈliːsn̩],"[ent, lie, ßen]",Deutsch,de,=== {{Wortart|Konjugierte Form|Deutsch}} ===\n...,
275581,454742,0,entließt,entlassen,True,[iːst],{'Verb': ['Konjugierte Form']},558,NaN,[ˌɛntˈliːst],"[ent, ließt]",Deutsch,de,=== {{Wortart|Konjugierte Form|Deutsch}} ===\n...,


time: 211 ms (started: 2023-02-15 16:17:20 +00:00)


## Reinigen

In [63]:
# bj_nlp.preprocess ausprobieren
bj_nlp.preprocess('Hälló?)()')

'Hällo?)()'

time: 54 ms (started: 2023-02-15 16:17:20 +00:00)


In [64]:
wiktionary.title = wiktionary.title.apply( lambda x: bj_nlp.preprocess(x)         ) #.copy()
wiktionary.lemma = wiktionary.lemma.apply( lambda x: bj_nlp.preprocess(x)         ) #.copy()

time: 21.2 s (started: 2023-02-15 16:17:20 +00:00)


## section_id2 und section_id

In [65]:
# section_id2: Das gleiche wie section_id, aber verbose
wiktionary['section_id2'] = wiktionary.title.str.replace(' ','_')

mask = (wiktionary.page_part > 0)
wiktionary.loc[mask,'section_id2'] = wiktionary[mask].section_id2 + '_' + wiktionary[mask].page_part.map(str)

time: 1.1 s (started: 2023-02-15 16:17:41 +00:00)


In [66]:
# Duplikate anzeigen (werden gleich gelöscht)
# Dups entstehen durch Akzente, die entfernt werden
dupmask = wiktionary.duplicated(subset=['section_id2'], keep=False)
result = wiktionary[dupmask].sort_values(['title','section_len'])
result = pak.drop_cols(result,['lang', 'lang_code','ipa','wikitext'])
mask = result.title.str.len() > 2
grid( result, mask, color='green' )

1370 rows out of 1433


,page_id,page_part,title,lemma,inflected,rhymes,pos,section_len,flexion,syllables,error,section_id2
795546,1062343,0,Aaners,Aaners,True,NaN,{'Deklinierte Form': []},275,NaN,"[Aa, ners’]",,Aaners
21304,53417,0,Aaners,Aaners,False,[aːnɐs],{'Substantiv': ['Vorname']},798,NaN,"[Aa, ners]",,Aaners
675729,937397,0,Aarhus,Aarhus,True,NaN,{'Deklinierte Form': []},278,NaN,"[Aar, hus’]",,Aarhus
106338,251980,0,Aarhus,Aarhus,False,NaN,{'Substantiv': ['Toponym']},1389,NaN,"[Aar, hus]",,Aarhus
738785,1003912,0,Aas,Aas,True,NaN,{'Deklinierte Form': []},259,NaN,[Aas’],,Aas
...,...,...,...,...,...,...,...,...,...,...,...,...
105047,250060,0,Äneas,Äneas,False,[eːas],"{'Substantiv': ['Vorname', 'Eigenname']}",2667,{'Genus': 'm'},"[Äne, as]",,Äneas
675824,937492,0,Äolus,Äolus,True,NaN,{'Deklinierte Form': []},272,NaN,"[Äo, lus’]",,Äolus
46234,117717,0,Äolus,Äolus,False,NaN,{'Substantiv': ['Eigenname']},947,{'Genus': 'm'},"[Äo, lus]",,Äolus
683360,945396,0,Ödipus,Ödipus,True,NaN,{'Deklinierte Form': []},277,NaN,"[Ödi, pus’]",,Ödipus


time: 422 ms (started: 2023-02-15 16:17:42 +00:00)


In [67]:
# Nach dem Reinigen sind die title nicht mehr eindeutig gewesen. 
# Wir verwenden nur die jeweils längste Seite.
# Danach ist section_id2 eindeutig.
len_pre = wiktionary.shape[0]
wiktionary = pak.rank(  wiktionary, cols_group=['section_id2'], col_score='section_len')
assert wiktionary.section_id2.is_unique
print( len_pre - wiktionary.shape[0],'Datensätze gelöscht')

rank: 721 rows less, now 841745 rows
721 Datensätze gelöscht
time: 3.07 s (started: 2023-02-15 16:17:43 +00:00)


In [68]:
# section_id 
wiktionary['section_id'] = wiktionary.page_id * 100 + wiktionary.page_part
assert wiktionary.section_id.is_unique

time: 102 ms (started: 2023-02-15 16:17:46 +00:00)


In [69]:
# Duplikate anzeigen
dupmask = wiktionary.duplicated(subset=['section_id'], keep=False)
grid( wiktionary[dupmask].sort_values('title'), color='green'   )

No rows
time: 135 ms (started: 2023-02-15 16:17:46 +00:00)


In [70]:
# Zeilen wieder sortieren
wiktionary = wiktionary.sort_values(['section_id'])

time: 569 ms (started: 2023-02-15 16:17:46 +00:00)


In [71]:
# Spalten in Ordnung bringen
wiktionary = pak.move_cols(wiktionary, 'section_id')
wiktionary = pak.move_cols(wiktionary, 'section_id2', to='page_part')

time: 683 ms (started: 2023-02-15 16:17:47 +00:00)


In [72]:
grid(wiktionary.head(100))

100 rows


,section_id,page_id,page_part,section_id2,title,lemma,inflected,rhymes,pos,section_len,flexion,ipa,syllables,lang,lang_code,wikitext,error
0,0,0,0,Hallo,Hallo,Hallo,False,[oː],{'Substantiv': []},1935,"{'Genus': 'n', 'Nominativ Singular': 'Hallo', ...",[haˈloː],"[Hal, lo]",Deutsch,de,"=== {{Wortart|Substantiv|Deutsch}}, {{n}} ===\...",
1,600,6,0,Subfamilia,Subfamilia,Subfamilia,False,NaN,{'Substantiv': []},1272,"{'Genus': 'f', 'Nominativ Singular': 'Subfamil...",NaN,"[Sub, fa, mi, lia]",Deutsch,de,"=== {{Wortart|Substantiv|Deutsch}}, {{f}} ===\...",
2,900,9,0,Subregnum,Subregnum,Subregnum,False,NaN,{'Substantiv': []},1277,"{'Genus': 'n', 'Nominativ Singular': 'Subregnu...",[zʊpˈʁeːɡnʊm],"[Sub, reg, num]",Deutsch,de,"=== {{Wortart|Substantiv|Deutsch}}, {{n}} ===\...",
3,1000,10,0,Subdivisio,Subdivisio,Subdivisio,False,NaN,{'Substantiv': []},1301,"{'Genus': 'f', 'Nominativ Singular': 'Subdivis...",[zʊpdiˈviːzi̯o],"[Sub, di, vi, sio]",Deutsch,de,"=== {{Wortart|Substantiv|Deutsch}}, {{f}} ===\...",
4,1100,11,0,Phylum,Phylum,Phylum,False,[yːlʊm],{'Substantiv': []},1940,"{'Genus': 'n', 'Nominativ Singular': 'Phylum',...",[ˈfyːlʊm],"[Phy, lum]",Deutsch,de,"=== {{Wortart|Substantiv|Deutsch}}, {{n}} ===\...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,16400,164,0,HI,HI,HI,False,NaN,{'Abkürzung': []},290,NaN,NaN,NaN,Deutsch,de,=== {{Wortart|Abkürzung|Deutsch}} ===\n\n{{Bed...,
96,16600,166,0,Boot,Boot,Boot,False,[oːt],{'Substantiv': []},9937,"{'Genus': 'n', 'Nominativ Singular': 'Boot', '...",[boːt],[Boot],Deutsch,de,"=== {{Wortart|Substantiv|Deutsch}}, {{n}} ===\...",
97,16601,166,1,Boot_1,Boot,Boot,False,[uːt],{'Substantiv': []},1603,"{'Genus': 'm', 'Nominativ Singular': 'Boot', '...",[buːt],[Boot],Deutsch,de,"=== {{Wortart|Substantiv|Deutsch}}, {{m}} ===\...",
98,16602,166,2,Boot_2,Boot,Boot,False,[oːt],{'Substantiv': ['Nachname']},2480,NaN,[boːt],[Boot],Deutsch,de,"=== {{Wortart|Substantiv|Deutsch}}, {{mf}}, {{...",


time: 87.7 ms (started: 2023-02-15 16:17:48 +00:00)


## Erste Korrekturen

In [73]:
mask1 = wiktionary.title == wiktionary.lemma
mask2A = wiktionary.pos.str.contains('Deklinierte Form')
mask2B = wiktionary.pos.str.contains('Konjugierte Form')
mask = mask1  &  (mask2A | mask2B)
wiktionary[mask]

,section_id,page_id,page_part,section_id2,title,lemma,inflected,rhymes,pos,section_len,flexion,ipa,syllables,lang,lang_code,wikitext,error


time: 3.31 s (started: 2023-02-15 16:17:48 +00:00)


## Fehler finden

In [74]:
# Fehler: title zu kurz
mask = (wiktionary.title.str.len() < 2)  
check_mask(wiktionary, mask, 0, 800)
wiktionary.loc[mask,'error'] = 'title zu kurz'
#wiktionary = pak.drop_rows(wiktionary,mask)

check_mask:  206 rows
time: 429 ms (started: 2023-02-15 16:17:51 +00:00)


In [75]:
# Fehler: page_part > 9
mask = (wiktionary.page_part > 9)   &   (wiktionary.error == '')
check_mask( wiktionary, mask, 0, 10 )
wiktionary.loc[mask,'error'] = 'page_part > 9'

check_mask:  0 rows
time: 117 ms (started: 2023-02-15 16:17:52 +00:00)


In [76]:
# Fehler finden: Uneindeutige Zuordnung
assert pak.same_but_different(wiktionary, ['page_id'], 'title').shape[0] == 0
assert pak.same_but_different(wiktionary, ['title'],   'page_id').shape[0] < 10  # es gibt ein paar Ausnahmen. Das kommt durch die Bereinigung. Ist aber egal.

time: 1.86 s (started: 2023-02-15 16:17:52 +00:00)


In [77]:
# Fehler finden: Uneindeutige Zuordnung
assert pak.same_but_different(wiktionary, ['section_id'],  'section_id2').shape[0] == 0
assert pak.same_but_different(wiktionary, ['section_id2'], 'section_id').shape[0] == 0

time: 2.41 s (started: 2023-02-15 16:17:54 +00:00)


In [78]:
# lang_code ist nan
mask = wiktionary.lang_code.isna()   &   (wiktionary.error == '')
check_mask(wiktionary, mask, 0, 9000)
wiktionary.loc[mask,'error'] = 'lang_code ist nan'
#grid(wiktionary, mask)

check_mask:  10 rows
time: 268 ms (started: 2023-02-15 16:17:56 +00:00)


In [79]:
# lang_code != de
mask = (wiktionary.lang_code != 'de')   &   (wiktionary.error == '')
check_mask(wiktionary, mask, 0, 10)
wiktionary.loc[mask,'error'] = 'lang_code != de'

check_mask:  0 rows
time: 318 ms (started: 2023-02-15 16:17:56 +00:00)


In [80]:
# lang ist nan
mask = wiktionary.lang.isna()   &   (wiktionary.error == '')
check_mask(wiktionary, mask, 0, 200)
wiktionary.loc[mask,'error'] = 'lang ist nan'
#grid(wiktionary, mask)

check_mask:  0 rows
time: 325 ms (started: 2023-02-15 16:17:57 +00:00)


In [81]:
# lang != Deutsch
mask = (wiktionary.lang != 'Deutsch')   &  (wiktionary.error == '')
check_mask(wiktionary, mask, 0, 10000)
wiktionary.loc[mask,'error'] = 'lang != Deutsch'

check_mask:  0 rows
time: 381 ms (started: 2023-02-15 16:17:57 +00:00)


In [82]:
# title zu viele Wörter
mask = (wiktionary.title.str.split().str.len() > 2)   &  (wiktionary.error == '')
check_mask(wiktionary, mask, 0, 4000)
wiktionary.loc[mask,'error'] = 'title zu viele Wörter'

check_mask:  3910 rows
time: 3.1 s (started: 2023-02-15 16:17:57 +00:00)


In [83]:
# lemma zu kurz
mask = (wiktionary.lemma.str.len() < 2)   &  (wiktionary.error == '')
check_mask(wiktionary, mask, 0, 100)
wiktionary.loc[mask,'error'] = 'lemma zu kurz'
#grid(wiktionary,mask)

check_mask:  26 rows
time: 485 ms (started: 2023-02-15 16:18:00 +00:00)


In [84]:
# pos ist nan
mask = wiktionary.pos.isna()   &   (wiktionary.error == '')
check_mask(wiktionary, mask, 0, 100)
wiktionary.loc[mask,'error'] = 'pos ist nan'
#grid(wiktionary, mask)

check_mask:  0 rows
time: 239 ms (started: 2023-02-15 16:18:01 +00:00)


In [85]:
# pos zu kurz
#mask = (wiktionary.pos.str.len() < 2)   &  wiktionary.error.isnull()
#check_mask(wiktionary, mask, 0, 10)
#wiktionary.loc[mask,'error'] = 'pos zu kurz'

time: 50.2 ms (started: 2023-02-15 16:18:01 +00:00)


## Trash

In [86]:
# Fehler aussortieren
mask = wiktionary.error != ''
wiktionary, wiktionary_trash = pak.move_rows( wiktionary, mask )       

Move 4152 rows from 841745
time: 762 ms (started: 2023-02-15 16:18:01 +00:00)


In [87]:
pak.analyse_freqs(wiktionary_trash.error)

,item,item_count,item_percent,graph
0,title zu viele Wörter,3910,94.2,###############################################
1,title zu kurz,206,5.0,##
2,lemma zu kurz,26,0.6,
3,lang_code ist nan,10,0.2,


time: 62.7 ms (started: 2023-02-15 16:18:02 +00:00)


In [88]:
# Bestimmten Fehler anzeigen
# mask = wiktionary_trash.error == 'lang_code ist nan'
mask = wiktionary_trash.error == 'lemma zu kurz'
grid(wiktionary_trash, mask)

26 rows out of 4152


,section_id,page_id,page_part,section_id2,title,lemma,inflected,rhymes,pos,section_len,flexion,ipa,syllables,lang,lang_code,wikitext,error,msg
10386,1712500,17125,0,as,as,a,True,[aːs],{'Deklinierte Form': []},491,NaN,[aːs],[as],Deutsch,de,=== {{Wortart|Deklinierte Form|Deutsch}} ===\n...,lemma zu kurz,0
12905,2100703,21007,3,As_3,As,A,True,[aːs],{'Deklinierte Form': []},534,NaN,[aːs],[As],Deutsch,de,=== {{Wortart|Deklinierte Form|Deutsch}} ===\n...,lemma zu kurz,0
12914,2103602,21036,2,Es_2,Es,E,True,[eːs],{'Deklinierte Form': []},453,NaN,[eːs],[Es],Deutsch,de,=== {{Wortart|Deklinierte Form|Deutsch}} ===\n...,lemma zu kurz,0
447222,66500500,665005,0,Ts,Ts,T,True,[eːs],{'Deklinierte Form': []},454,NaN,[teːs],[Ts],Deutsch,de,=== {{Wortart|Deklinierte Form|Deutsch}} ===\n...,lemma zu kurz,0
480370,71223700,712237,0,Xen,Xen,X,True,[ɪksn̩],{'Deklinierte Form': []},286,NaN,[ˈɪksn̩],[Xen],Deutsch,de,=== {{Wortart|Deklinierte Form|Deutsch}} ===\n...,lemma zu kurz,0
480396,71226400,712264,0,cs,cs,c,True,[eːs],{'Deklinierte Form': []},456,NaN,[t͡seːs],[cs],Deutsch,de,=== {{Wortart|Deklinierte Form|Deutsch}} ===\n...,lemma zu kurz,0
480408,71227700,712277,0,fs,fs,f,True,NaN,{'Deklinierte Form': []},429,NaN,[ɛfs],[fs],Deutsch,de,=== {{Wortart|Deklinierte Form|Deutsch}} ===\n...,lemma zu kurz,0
480427,71229600,712296,0,gs,gs,g,True,[eːs],{'Deklinierte Form': []},462,NaN,[ɡeːs],[gs],Deutsch,de,=== {{Wortart|Deklinierte Form|Deutsch}} ===\n...,lemma zu kurz,0
480428,71229700,712297,0,hs,hs,h,True,[aːs],{'Deklinierte Form': []},462,NaN,[haːs],[hs],Deutsch,de,=== {{Wortart|Deklinierte Form|Deutsch}} ===\n...,lemma zu kurz,0
643845,90388000,903880,0,Bs,Bs,B,True,[eːs],{'Deklinierte Form': []},484,NaN,[beːs],[Bs],Deutsch,de,=== {{Wortart|Deklinierte Form|Deutsch}} ===\n...,lemma zu kurz,0


time: 97.4 ms (started: 2023-02-15 16:18:02 +00:00)


## nnan

In [89]:
pak.nnan(wiktionary)

rhymes       527175
flexion      713384
ipa           14004
syllables      4949
dtype: int64

time: 951 ms (started: 2023-02-15 16:18:02 +00:00)


## Spalten löschen

In [90]:
# lang, lang_code, ipa, syllables
assert wiktionary.lang.nunique() == 1
assert wiktionary.lang_code.nunique() == 1
#wiktionary = pak.drop_cols(wiktionary,      ['lang','lang_code','inflected'])

time: 309 ms (started: 2023-02-15 16:18:03 +00:00)


## Speichern

In [91]:
# Keine Leerzeichen in der ID
mask = wiktionary.section_id2.str.contains(' ')
check_mask(wiktionary,mask,0)

check_mask:  0 rows
time: 406 ms (started: 2023-02-15 16:18:03 +00:00)


In [92]:
if wiktionary_filename:
    pak.dump_pickle(wiktionary, wiktionary_filename) 

time: 32.2 s (started: 2023-02-15 16:18:04 +00:00)


In [93]:
if wiktionary_trash_filename   and   wiktionary_trash.shape[0] > 0:
    pak.dump_pickle(wiktionary_trash, wiktionary_trash_filename)    

time: 232 ms (started: 2023-02-15 16:18:36 +00:00)


In [105]:
# Save in Google Drive
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av data_00 drive/MyDrive/data_00  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'data_00' -> 'drive/MyDrive/data_00/data_00'
'data_00/dewiktionary-latest-pages-articles-multistream.xml.bz2' -> 'drive/MyDrive/data_00/data_00/dewiktionary-latest-pages-articles-multistream.xml.bz2'
'data_00/wiktionary_short.pickle' -> 'drive/MyDrive/data_00/data_00/wiktionary_short.pickle'
'data_00/wiktionary_trash_short.pickle' -> 'drive/MyDrive/data_00/data_00/wiktionary_trash_short.pickle'
'data_00/wiktionary_all.pickle' -> 'drive/MyDrive/data_00/data_00/wiktionary_all.pickle'
'data_00/wiktionary_trash_all.pickle' -> 'drive/MyDrive/data_00/data_00/wiktionary_trash_all.pickle'
time: 6.9 s (started: 2023-02-15 16:36:47 +00:00)


## Kontrolle

In [95]:
# suche_debug
mask = wiktionary.title.isin(suche_debug)
grid(wiktionary, mask)

1 rows out of 837593


,section_id,page_id,page_part,section_id2,title,lemma,inflected,rhymes,pos,section_len,flexion,ipa,syllables,lang,lang_code,wikitext,error
14147,2270300,22703,0,entlassen,entlassen,entlassen,False,[asn̩],{'Verb': []},4961,"{'Präsens_ich': 'entlasse', 'Präsens_du': 'ent...",[ɛntˈlasn̩],"[ent, las, sen]",Deutsch,de,=== {{Wortart|Verb|Deutsch}} ===\n\n{{Deutsch ...,


time: 119 ms (started: 2023-02-15 16:18:36 +00:00)


In [96]:
# histo section_len
a = (wiktionary.section_len / 10000).round().astype('int') * 1000
pak.analyse_freqs(a, sort_count=False)

,item,item_count,item_percent,graph
0,0,830604,99.2,#################################################
1,1000,6701,0.8,
2,2000,223,0.0,
3,3000,37,0.0,
4,4000,12,0.0,
5,5000,7,0.0,
6,6000,3,0.0,
7,7000,3,0.0,
8,8000,2,0.0,
9,9000,1,0.0,


time: 73.6 ms (started: 2023-02-15 16:18:37 +00:00)


In [97]:
# page_part
pak.analyse_freqs(wiktionary.page_part, sort_count=False)

,item,item_count,item_percent,graph
0,0,802084,95.8,###############################################
1,1,33817,4.0,##
2,2,1452,0.2,
3,3,205,0.0,
4,4,29,0.0,
5,5,5,0.0,
6,6,1,0.0,


time: 72.8 ms (started: 2023-02-15 16:18:37 +00:00)


In [98]:
raise Stop

Stop Time:  16:18:37
Elapsed:    18 mins, 9 secs


Stop: ignored

time: 59.7 ms (started: 2023-02-15 16:18:37 +00:00)
